# Atualizar Mapa Cidades

In [ ]:
import pandas as pd
import json
import glob

## Ler todos os CSV

In [43]:
##### Ler arquivos CSV

# IDHM buscar por Cidade:

df_idhm = pd.read_csv(r"C:\Users\samfs\Documents\GITHUB\Mapas_Brasil\Populacao,PIB,IDHM\IDHM_Cidades.csv")
df_idhm.head()

for index,row in df_idhm.iterrows():
    municipio = str(row['municipio']).split('(')[0]
    row['municipio'] = municipio

df_idhm.head(20)

,id,municipio,IDHM,IDHM Renda,IDHM Longevidade,IDHM Educacao
0,1,São Caetano do Sul,"0,862","0,891","0,887","0,811"
1,2,Águas de São Pedro,"0,854","0,849","0,890","0,825"
2,3,Florianópolis,"0,847","0,870","0,873","0,800"
3,4,Balneário Camboriú,"0,845","0,854","0,894","0,789"
4,4,Vitória,"0,845","0,876","0,855","0,805"
5,6,Santos,"0,840","0,861","0,852","0,807"
6,7,Niterói,"0,837","0,887","0,854","0,773"
7,8,Joaçaba,"0,827","0,823","0,891","0,771"
8,9,Brasília,"0,824","0,863","0,873","0,742"
9,10,Curitiba,"0,823","0,850","0,855","0,768"


In [9]:
# Percentile buscar por Cidade:

df_percentile = pd.read_csv(r"C:\Users\samfs\Documents\GITHUB\Mapas_Brasil\Populacao,PIB,IDHM\PercentilIdade_Municipios.csv")

df_percentile.head()

,Cod_Municipo,UF,Municipio,0a5,6a14,15a24,25a39,40a59,40a59.1,60
0,4200051,SC,Abdon Batista,2653,7.84,13.87,16.02,19.68,28.42,14.17
1,1700251,TO,Abreulândia,2391,12.05,19.49,14.72,23.21,20.91,9.62
2,3100401,MG,Acaiaca,3920,7.96,15.36,16.68,22.24,22.60,15.15
3,4300034,RS,Aceguá,4394,7.62,16.23,15.34,21.46,25.83,13.52
4,5200159,GO,Adelândia,2477,7.59,13.32,16.92,22.61,25.80,13.77


In [20]:
# PIB buscar por Cidade:

df_pib = pd.read_csv(r"C:\Users\samfs\Documents\GITHUB\Mapas_Brasil\Populacao,PIB,IDHM\PIB_Municipios.csv")
df_pib.head()

df_pib.columns = ['Municipio','Cod_Municipio', 'Atv3', 'Atv2', 'Atv1', 'PIBpc', 'PIB' ]
df_pib.head()

,Municipio,Cod_Municipio,Atv3,Atv2,Atv1,PIBpc,PIB
0,Rondônia,1100015,Demais serviços,"Pecuária, inclusive apoio à pecuária","Administração, defesa, educação e saúde públic...",10731.18,262076.878
1,Rondônia,1100023,Comércio e reparação de veículos automotores e...,Demais serviços,"Administração, defesa, educação e saúde públic...",15103.86,1364693.780
2,Rondônia,1100031,Demais serviços,"Pecuária, inclusive apoio à pecuária","Administração, defesa, educação e saúde públic...",11033.62,69611.114
3,Rondônia,1100049,Comércio e reparação de veículos automotores e...,Demais serviços,"Administração, defesa, educação e saúde públic...",15095.15,1186494.091
4,Rondônia,1100056,Comércio e reparação de veículos automotores e...,Demais serviços,"Administração, defesa, educação e saúde públic...",13037.06,222021.189


In [13]:
# PIB buscar por Cidade:

df_censo = pd.read_csv(r"C:\Users\samfs\Documents\GITHUB\Mapas_Brasil\Populacao,PIB,IDHM\UrbanoxRuralxSexo_Municipios.csv")
df_censo.head()

,Cod_Municipo,UF,Municipio,Total,Urbano,Rural,Homem,Mulher,Razao
0,4200051,SC,Abdon Batista,2653,27.29,72.71,52.09,47.91,108.724692
1,1700251,TO,Abreulândia,2391,61.73,38.27,52.74,47.26,111.595430
2,3100401,MG,Acaiaca,3920,65.13,34.87,48.32,51.68,93.498452
3,4300034,RS,Aceguá,4394,24.10,75.9,52.41,47.59,110.128178
4,5200159,GO,Adelândia,2477,89.10,10.9,51.39,48.61,105.718988


------------------------------------------------------------------------------------------------------------------------------

## Abrir o GeoJSON dos Municipios

In [64]:

def get_idhm(nome_cidade):
    nome_cidade = str(nome_cidade) +  ' '
    df_idhm_cidade = df_idhm[df_idhm.municipio == nome_cidade]

    # Caso o muniicpio nao tenha idhm registrado na minha planilha
    if(df_idhm_cidade.shape[0] == 0 ):
         return({})
    else:
        dict_idhm = {
            "IDHM": str(df_idhm_cidade.iloc[0][2]),
            "IDHM Renda": str(df_idhm_cidade.iloc[0][3]),
            "IDHM Longevidade": str(df_idhm_cidade.iloc[0][4]),
            "IDHM Educacao": str(df_idhm_cidade.iloc[0][5])
        }
        return(dict_idhm)
    
def get_percentil(cod):
    df_percentile_cidade = df_percentile[df_percentile.Cod_Municipo == cod]

    # Caso o muniicpio nao tenha idhm registrado na minha planilha
    if(df_percentile_cidade.shape[0] == 0 ):
         return({})
    else:
        dict_percentil = {
            "0 a 5 Anos": str(df_percentile_cidade.iloc[0][4]),
            "6 a 14 Anos": str(df_percentile_cidade.iloc[0][5]),
            "15 a 24 Anos": str(df_percentile_cidade.iloc[0][6]),
            "25 a 39 Anos": str(df_percentile_cidade.iloc[0][7]),
            "40 a 59 Anos": str(df_percentile_cidade.iloc[0][8]),
            "60 Anos mais": str(df_percentile_cidade.iloc[0][9])
        }
        return(dict_percentil)    
    
def get_pib(cod):
    df_pib_cidade = df_pib[df_pib.Cod_Municipio == cod]
    
    # Caso o muniicpio nao tenha pib registrado na minha planilha
    if(df_pib_cidade.shape[0] == 0 ):
         return({})
    else:
        dict_pib = {
            "Terceira Atividade de Maior Renda": str(df_pib_cidade.iloc[0][2]),
            "Segunda Atividade de Maior Renda": str(df_pib_cidade.iloc[0][3]),
            "Primeira Atividade de Maior Renda": str(df_pib_cidade.iloc[0][4]),
            "PIBpc": str(df_pib_cidade.iloc[0][5]),
            "PIB": str(df_pib_cidade.iloc[0][6]),

        }
        return(dict_pib)

    
def get_censo(cod):
    df_censo_cidade = df_censo[df_censo.Cod_Municipo == cod]
    
    # Caso o muniicpio nao tenha censo registrado na minha planilha
    if(df_censo_cidade.shape[0] == 0 ):
         return({})
    else:
        dict_censo = {
            "Populacao Urbana(%)": str(df_censo_cidade.iloc[0][4]),
            "Populacao Rural(%)": str(df_censo_cidade.iloc[0][5]),
            "Homem(%)": str(df_censo_cidade.iloc[0][6]),
            "Mulher(%)": str(df_censo_cidade.iloc[0][7]),
        }
        return(dict_censo)  
    
novo_json = []

def sigla_ibge(cod):
    dic_estado = {"53":"DF","11":"RO", "12":"AC", "13":"AM", "14":"RR", "15":"PA", "16":"AP", "17":"TO", "21":"MA", "22":"PI" , "23":"CE" , "24":"RN" , "25":"PB" , "26":"PE", "27":"AL" , "28":"SE", "29":"BA", "31":"MG", "32":"ES" , "33":"RJ", "35":"SP", "41":"PR", "42":"SC", "43":"RS", "50":"MS", "51":"MT", "52":"GO"}
    return(dic_estado[cod])


with open(r'C:\Users\samfs\Documents\GITHUB\Mapas_Brasil\Mapas\Municipios.geojson', encoding='utf-8') as json_file:
    data = json.load(json_file)
    
    for feat in data['features']:
        municipio = str(feat['properties']['Municipio'])
        cod_municipio = int(feat['properties']['Codigo_Municipio'])
        
        dict_completo = {}
        # Idhm
        dict_idhm = get_idhm(municipio)
        dict_completo.update(dict_idhm)
        # Percentil
        dict_percentil = get_percentil(cod_municipio)
        dict_completo.update(dict_percentil)
        # PIB
        dict_pib = get_pib(cod_municipio)
        dict_completo.update(dict_pib)
        # Censo
        dict_censo = get_censo(cod_municipio)
        dict_completo.update(dict_censo)
        
        properties = {
            "UF": feat['properties']['UF'],
            "Estado": feat['properties']['Estado'],
            "Cod_Estado": str(sigla_ibge(feat['properties']['UF'])),
            "Municipio": feat['properties']['Municipio'],
            "Codigo_Municipio": feat['properties']['Codigo_Municipio'],
            "Populacao": feat['properties']['Populacao']
        }
        
        properties.update(dict_completo)
       
        
        doc = {
            "type": "Feature",
            "properties": properties,
            "geometry": {
                "type": feat['geometry']['type'],
                "coordinates": feat['geometry']['coordinates']
            }          
        }
        novo_json.append(doc)
        
        
print("Fim")
##### Salvar arquivo
name_file = 'Municipios_Completo.geojson'
geojson = {
    "type":"FeatureCollection",
    "features": novo_json
}
with open(name_file, 'w') as json_file:
    json.dump(geojson, json_file, ensure_ascii=True)
    

Fim
